In [127]:
using JuMP
using SDPA
using SCIP
using LinearAlgebra
using SCS

# INFEASIBILITY CERTIFICATE

### PRIMAL

In [128]:
model_primal = Model(SCS.Optimizer)

A JuMP Model
├ solver: SCS
├ objective_sense: FEASIBILITY_SENSE
├ num_variables: 0
├ num_constraints: 0
└ Names registered in the model: none

In [129]:
@variable(model_primal, X[1:2,1:2], PSD)

2×2 Symmetric{VariableRef, Matrix{VariableRef}}:
 X[1,1]  X[1,2]
 X[1,2]  X[2,2]

In [130]:
@objective(model_primal, Min, 0)

0

In [131]:
@constraint(model_primal, X[1,1] == 1)
@constraint(model_primal, X[2,2] == -1)

X[2,2] == -1

In [132]:
optimize!(model_primal)

------------------------------------------------------------------
	       SCS v3.2.8 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
------------------------------------------------------------------
problem:  variables n: 3, constraints m: 5
cones: 	  z: primal zero / dual free vars: 2
	  s: psd vars: 3, ssize: 1
settings: eps_abs: 1.0e-004, eps_rel: 1.0e-004, eps_infeas: 1.0e-007
	  alpha: 1.50, scale: 1.00e-001, adaptive_scale: 1
	  max_iters: 100000, normalize: 1, rho_x: 1.00e-006
	  acceleration_lookback: 10, acceleration_interval: 10
	  compiled with openmp parallelization enabled
lin-sys:  sparse-direct-amd-qdldl
	  nnz(A): 5, nnz(P): 0
------------------------------------------------------------------
 iter | pri res | dua res |   gap   |   obj   |  scale  | time (s)
------------------------------------------------------------------
     0|2.00e+000 1.58e-001 3.16e-001 1.58e-001 1.00e-001 5.11e-004 
    50|1.66e+005 1.28e+004 1.98e+018 9.89e+017 1.0

### PRIMAL IS INFEASIBLE

In [133]:
termination_status(model_primal)

INFEASIBLE::TerminationStatusCode = 2

## DUAL


In [134]:
model_dual = Model(SCS.Optimizer)

A JuMP Model
├ solver: SCS
├ objective_sense: FEASIBILITY_SENSE
├ num_variables: 0
├ num_constraints: 0
└ Names registered in the model: none

In [135]:
@variable(model_dual, Y1)
@variable(model_dual, Y2)

Y2

In [136]:
@objective(model_dual, Max, Y1 - Y2)

Y1 - Y2

In [137]:
@constraint(model_dual, Y1 <= 0)
@constraint(model_dual, Y2 <= 0)

Y2 <= 0

In [138]:
optimize!(model_dual)

------------------------------------------------------------------
	       SCS v3.2.8 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
------------------------------------------------------------------
problem:  variables n: 2, constraints m: 2
cones: 	  l: linear vars: 2
settings: eps_abs: 1.0e-004, eps_rel: 1.0e-004, eps_infeas: 1.0e-007
	  alpha: 1.50, scale: 1.00e-001, adaptive_scale: 1
	  max_iters: 100000, normalize: 1, rho_x: 1.00e-006
	  acceleration_lookback: 10, acceleration_interval: 10
	  compiled with openmp parallelization enabled
lin-sys:  sparse-direct-amd-qdldl
	  nnz(A): 2, nnz(P): 0
------------------------------------------------------------------
 iter | pri res | dua res |   gap   |   obj   |  scale  | time (s)
------------------------------------------------------------------
     0|2.00e+001 1.00e+000 4.00e+001 -2.00e+001 1.00e-001 1.69e-004 
    50|5.22e+003 0.00e+000 9.95e+017 -4.97e+017 1.00e-001 3.45e-004 
-------------------------

### DUAL IS UNBOUNDED

In [139]:
termination_status(model_dual)

DUAL_INFEASIBLE::TerminationStatusCode = 3

# STRONG DUALITY FOR LOVASZ THETA SDP

In [140]:
n = 5
edges = [(1,2),(2,3),(3,4),(4,5),(5,1)]


5-element Vector{Tuple{Int64, Int64}}:
 (1, 2)
 (2, 3)
 (3, 4)
 (4, 5)
 (5, 1)

In [141]:
model = Model(SCS.Optimizer)
set_silent(model)

@variable(model, X[1:n, 1:n], PSD)

# Objective: maximize ⟨J, X⟩
J = ones(n, n)
@objective(model, Max, sum(J[i,j] * X[i,j] for i in 1:n, j in 1:n))

# Trace constraint
@constraint(model, trace, sum(X[i,i] for i in 1:n) == 1)

# Edge constraints
edge_con = Dict()
for (i,j) in edges
    edge_con[(i,j)] = @constraint(model, X[i,j] == 0)
    edge_con[(j,i)] = @constraint(model, X[j,i] == 0)
end

optimize!(model)


In [147]:
dual_model = Model(SCS.Optimizer)

@variable(dual_model, y)
@variable(dual_model, Z[1:n,1:n], PSD)

@objective(dual_model, Min, y)

for i in 1:n
    @constraint(dual_model, Z[i,i] == y - 1)
end
for i in 1:n
    for j in i+1:n
        if !adj[i,j]
            @constraint(dual_model, Z[i,j] == -1)
            @constraint(dual_model, Z[j,i] == -1)
        end
    end
end

optimize!(dual_model)

------------------------------------------------------------------
	       SCS v3.2.8 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
------------------------------------------------------------------
problem:  variables n: 16, constraints m: 30
cones: 	  z: primal zero / dual free vars: 15
	  s: psd vars: 15, ssize: 1
settings: eps_abs: 1.0e-004, eps_rel: 1.0e-004, eps_infeas: 1.0e-007
	  alpha: 1.50, scale: 1.00e-001, adaptive_scale: 1
	  max_iters: 100000, normalize: 1, rho_x: 1.00e-006
	  acceleration_lookback: 10, acceleration_interval: 10
	  compiled with openmp parallelization enabled
lin-sys:  sparse-direct-amd-qdldl
	  nnz(A): 35, nnz(P): 0
------------------------------------------------------------------
 iter | pri res | dua res |   gap   |   obj   |  scale  | time (s)
------------------------------------------------------------------
     0|5.36e+000 1.00e+000 6.86e+000 6.51e-002 1.00e-001 2.13e-004 
    50|1.85e-007 2.93e-008 6.02e-007 2.24e+00

In [149]:
pobj = objective_value(model)
dobj = objective_value(dual_model)

println("Primal objective = ", pobj)
println("Dual objective   = ", dobj)

d_gap = abs(pobj - dobj)

println("Duality gap      = ", d_gap)


Primal objective = 2.2360675084359767
Dual objective   = 2.236068299572306
Duality gap      = 7.911363293366946e-7


# Complementary slackness

In [151]:
Zval = value.(Z)

cs = tr(Xval * Zval)
println("Complementary slackness ⟨X,Z⟩ = ", cs)


Complementary slackness ⟨X,Z⟩ = 7.989029525265374e-7


In [156]:
# 0.000000
factor = 10.0^6

result = floor(cs * factor) / factor
result

0.0